Note: part of the training code is based off tensorflows text generation example here: https://www.tensorflow.org/text/tutorials/text_generation. However we make drastic changes to make it compatible for our data and for word generation instead of character generation and rhyme scheme implementation.

In [1]:
import tensorflow as tf
import keras
import numpy as np
import string
from keras.preprocessing.text import Tokenizer
import pickle

2022-12-10 16:24:02.203320: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-10 16:24:02.284267: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-10 16:24:02.676867: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/x86_64-linux-gnu/gazebo-11/plugins:/opt/ros/foxy/opt/yaml_cpp_vendor/lib:/opt/

In [2]:
# File path for training data
training_file_path = "poems.txt"

f = open(training_file_path, "r", encoding="utf-8")
content = f.read()
f.close()

# lower case and remove punctuation
content = content.lower()
content_no_punctuation = ""
i = 0

while i < (len(content)):
  if content[i] not in string.punctuation:
    content_no_punctuation += content[i]
  i += 1

content = content_no_punctuation


# Split data into array
lines = content.splitlines()
data = []
for l in lines:
  sentence = l.split()
  sentence.append('\n')
  data.append(sentence)
  
# Train tokenizer and encode data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data)
encoded = tokenizer.texts_to_sequences(data)

flatten_list = [j for sub in encoded for j in sub]
vocab = sorted(set(tokenizer.word_docs))

In [3]:
ids_dataset = tf.data.Dataset.from_tensor_slices(flatten_list)

# Create batches of sequence size 200
seq_length = 200

sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

2022-12-10 16:24:03.798030: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-10 16:24:03.801206: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-10 16:24:03.801299: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-10 16:24:03.801613: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operati

In [4]:
# Here we do the n gram split to get gold labels from our training data
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [5]:
# Batch size
BATCH_SIZE = 32

dataset = (
    dataset
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

vocab_size = len(tokenizer.word_index) + 1

# Embedding size
embedding_dim = 512

# RNN size
rnn_units = 1024

In [6]:
class Model(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [7]:
model = Model(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [8]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [9]:
model.compile(optimizer='adam', loss=loss)

In [10]:
EPOCHS = 100
# Train here
history = model.fit(dataset, epochs=EPOCHS)

Epoch 1/100


2022-12-10 16:24:05.837338: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2022-12-10 16:24:06.281663: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2022-12-10 16:24:06.349396: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f420b585120 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2022-12-10 16:24:06.349412: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Ti Laptop GPU, Compute Capability 8.6
2022-12-10 16:24:06.352384: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2022-12-10 16:24:06.399131: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is ol

 1/27 [>.............................] - ETA: 1:05 - loss: 9.4208

2022-12-10 16:24:06.629828: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2022-12-10 16:24:06.776679: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2022-12-10 16:24:06.779747: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-addr

 2/27 [=>............................] - ETA: 9s - loss: 9.4174  

...

2022-12-10 16:24:15.647516: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


26/27 [===========================>..] - ETA: 0s - loss: 7.3383

2022-12-10 16:24:16.007008: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


27/27 [==============================] - 12s 366ms/step - loss: 7.3154
Epoch 2/100
27/27 [==============================] - 10s 354ms/step - loss: 6.5787
Epoch 3/100
27/27 [==============================] - 10s 355ms/step - loss: 6.4275
Epoch 4/100
27/27 [==============================] - 10s 355ms/step - loss: 6.1749
Epoch 5/100
27/27 [==============================] - 10s 356ms/step - loss: 6.0104
Epoch 6/100
27/27 [==============================] - 10s 358ms/step - loss: 5.8823
Epoch 7/100
27/27 [==============================] - 10s 357ms/step - loss: 5.7617
Epoch 8/100
27/27 [==============================] - 10s 357ms/step - loss: 5.6437
Epoch 9/100
27/27 [==============================] - 10s 357ms/step - loss: 5.5354
Epoch 10/100
27/27 [==============================] - 10s 357ms/step - loss: 5.4327
Epoch 11/100
27/27 [==============================] - 10s 357ms/step - loss: 5.3298
Epoch 12/100
27/27 [==============================] - 10s 357ms/step - loss: 5.2249
Epoch 13/100


KeyboardInterrupt: 

In [11]:
model.save_weights('model_poems')

with open('tokenizer', 'wb') as tokenizer_file:
  pickle.dump(tokenizer, tokenizer_file)